### High-level modeling plan

In 03_modeling.ipynb, we’ll:

- Load preprocessed data

- Set up MLflow experiment

- Each model run = one MLflow run

- Log params, metrics, and model artifact

- Train and compare multiple models

- Compare on validation metrics roc_auc, auc_pr (PR-AUC), f1, precision, recall


In [1]:
import time
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from scipy import sparse
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.ensemble import (
    AdaBoostClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    confusion_matrix,
    precision_recall_curve,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [2]:
class Config:
    features_dir = Path("../data/features")

In [3]:
X_train = sparse.load_npz(Config.features_dir / "X_train.npz")
X_val = sparse.load_npz(Config.features_dir / "X_val.npz")
X_test = sparse.load_npz(Config.features_dir / "X_test.npz")

X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

y_train = pd.read_csv(Config.features_dir / "y_train.csv").iloc[:, 0].values
y_val = pd.read_csv(Config.features_dir / "y_val.csv").iloc[:, 0].values
y_test = pd.read_csv(Config.features_dir / "y_test.csv").iloc[:, 0].values

print("X shapes:", X_train.shape, X_val.shape, X_test.shape)
print("y shapes:", y_train.shape, y_val.shape, y_test.shape)
print("Train class balance:", np.bincount(y_train))

X shapes: (3640, 174) (780, 174) (780, 174)
y shapes: (3640,) (780,) (780,)
Train class balance: [3500  140]


In [4]:
def evaluate_model(model, X, y):
    """Compute ROC-AUC, AUC-PR, precision, recall, F1."""
    if hasattr(model, "predict_proba"):
        scores = model.predict_proba(X)[:, 1]
    elif hasattr(model, "decision_function"):
        scores = model.decision_function(X)
    else:
        scores = model.predict(X)

    y_pred = (scores >= 0.5).astype(int) if scores.ndim == 1 else model.predict(X)

    roc_auc = roc_auc_score(y, scores)
    pr_auc = average_precision_score(y, scores)
    precision, recall, f1_val, _ = precision_recall_fscore_support(y, y_pred, average="binary", zero_division=0)
    return {
        "roc_auc": roc_auc,
        "auc_pr": pr_auc,
        "precision": precision,
        "recall": recall,
        "f1": f1_val,
    }

In [5]:
import mlflow


def log_diagnostic_plots(model, X, y, model_name: str):
    if hasattr(model, "predict_proba"):
        scores = model.predict_proba(X)[:, 1]
    elif hasattr(model, "decision_function"):
        scores = model.decision_function(X)
    else:
        scores = model.predict(X)

    y_pred = (scores >= 0.5).astype(int) if scores.ndim == 1 else model.predict(X)

    fpr, tpr, _ = roc_curve(y, scores)
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y, scores):.3f}")
    ax.plot([0, 1], [0, 1], "k--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(f"ROC Curve – {model_name}")
    ax.legend(loc="lower right")
    mlflow.log_figure(fig, f"plots/{model_name}_roc_curve.png")
    plt.close(fig)

    # PR curve
    precision, recall, _ = precision_recall_curve(y, scores)
    fig, ax = plt.subplots()
    ax.plot(recall, precision, label=f"AP = {average_precision_score(y, scores):.3f}")
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    ax.set_title(f"Precision–Recall Curve – {model_name}")
    ax.legend(loc="lower left")
    mlflow.log_figure(fig, f"plots/{model_name}_pr_curve.png")
    plt.close(fig)

    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(f"Confusion Matrix – {model_name}")
    fig.colorbar(im, ax=ax)
    ax.set_xticks([0, 1])
    ax.set_xticklabels(["Non-customer", "Customer"])
    ax.set_yticks([0, 1])
    ax.set_yticklabels(["Non-customer", "Customer"])
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], "d"),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )
    ax.set_ylabel("True label")
    ax.set_xlabel("Predicted label")
    mlflow.log_figure(fig, f"plots/{model_name}_confusion_matrix.png")
    plt.close(fig)

In [6]:
preprocessor = joblib.load(Config.features_dir / "preprocessor.pkl")
feature_names = preprocessor.get_feature_names_out()

In [7]:
import pandas as pd


def log_feature_importances(model, feature_names, model_name: str, top_k: int = 25):
    if not hasattr(model, "feature_importances_"):
        return

    importances = model.feature_importances_
    if importances is None or len(importances) != len(feature_names):
        return

    fi = (
        pd.DataFrame({"feature": feature_names, "importance": importances})
        .sort_values("importance", ascending=False)
        .head(top_k)
    )

    # Plot
    fig, ax = plt.subplots(figsize=(8, max(4, 0.3 * top_k)))
    ax.barh(fi["feature"], fi["importance"])
    ax.invert_yaxis()
    ax.set_title(f"Top {top_k} Feature Importances – {model_name}")
    ax.set_xlabel("Importance")
    plt.tight_layout()
    mlflow.log_figure(fig, f"feature_importances/{model_name}_feature_importances.png")
    plt.close(fig)

    # CSV as text artifact
    csv_str = fi.to_csv(index=False)
    mlflow.log_text(csv_str, f"feature_importances/{model_name}_feature_importances.csv")

In [8]:
experiment_name = "hubspot_prospect_conversion_gridsearch_v1"
mlflow.set_experiment(experiment_name)
print("Using MLflow experiment:", experiment_name)

/Users/ashwin/Applications/Master/venv/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/08 17:46:24 INFO mlflow.tracking.fluent: Experiment with name 'hubspot_prospect_conversion_gridsearch_v1' does not exist. Creating a new experiment.


Using MLflow experiment: hubspot_prospect_conversion_gridsearch_v1


In [9]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos

In [10]:
models_grids = {
    "LogisticRegression": (
        LogisticRegression(max_iter=2000, solver="lbfgs"),
        {
            "C": [0.1, 1.0, 10.0],
            "class_weight": ["balanced", None],
        },
    ),
    "SVC": (
        SVC(probability=True),
        {
            "C": [0.1, 1.0],
            "kernel": ["linear", "rbf"],
            "class_weight": ["balanced", None],
        },
    ),
    "KNeighborsClassifier": (
        KNeighborsClassifier(),
        {
            "n_neighbors": [3, 5, 7],
            "weights": ["uniform", "distance"],
        },
    ),
    "DecisionTreeClassifier": (
        DecisionTreeClassifier(random_state=42),
        {
            "max_depth": [3, 5, None],
            "class_weight": ["balanced", None],
        },
    ),
    "RandomForestClassifier": (
        RandomForestClassifier(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200],
            "max_depth": [3, 5, None],
            "class_weight": ["balanced", None],
        },
    ),
    "ExtraTreesClassifier": (
        ExtraTreesClassifier(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200],
            "max_depth": [3, 5, None],
            "class_weight": ["balanced", None],
        },
    ),
    "AdaBoostClassifier": (
        AdaBoostClassifier(random_state=42),
        {
            "n_estimators": [50, 100],
            "learning_rate": [0.05, 0.1, 0.5],
        },
    ),
    "GradientBoostingClassifier": (
        GradientBoostingClassifier(random_state=42),
        {
            "n_estimators": [50, 100],
            "learning_rate": [0.05, 0.1],
            "max_depth": [2, 3],
        },
    ),
    "XGBClassifier": (
        XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            use_label_encoder=False,
            random_state=42,
            n_jobs=-1,
        ),
        {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "max_depth": [3, 4],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
            "scale_pos_weight": [scale_pos_weight],
        },
    ),
    "GaussianNB": (
        GaussianNB(),
        {},  # no grid
    ),
    "LinearDiscriminantAnalysis": (
        LinearDiscriminantAnalysis(),
        {},  # simple baseline
    ),
    "QuadraticDiscriminantAnalysis": (
        QuadraticDiscriminantAnalysis(),
        {},  # can sometimes fail; we’ll catch errors
    ),
}

models_grids["CatBoostClassifier"] = (
    CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=False,
        scale_pos_weight=scale_pos_weight,
    ),
    {
        "depth": [4, 6],
        "learning_rate": [0.05, 0.1],
        "iterations": [200, 400],
    },
)

models_grids["LGBMClassifier"] = (
    LGBMClassifier(
        objective="binary",
        random_state=42,
        n_jobs=-1,
    ),
    {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, -1],
        "num_leaves": [15, 31],
        "class_weight": ["balanced", None],
    },
)

In [13]:
def run_gridsearch_experiment(
    name,
    model,
    param_grid,
    X_train,
    y_train,
    X_val,
    y_val,
    feature_names,
    cv_splits=5,
):
    """
    Runs GridSearchCV(KFold) → logs:
      - best params
      - CV mean/std AUC-PR
      - validation metrics
      - ROC/PR/confusion plots
      - feature importances (if available)
      - training time
    """
    print(f"\n=== Running GridSearch for {name} ===")
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

    with mlflow.start_run(run_name=f"{name}_gridsearch"):
        start_time = time.time()

        gs = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring="average_precision",  # AUC-PR for imbalance
            cv=kf,
            n_jobs=-1,
            refit=True,
            verbose=0,
        )

        gs.fit(X_train, y_train)

        elapsed = time.time() - start_time
        best_model = gs.best_estimator_
        cv_best_score = gs.best_score_
        cv_results = pd.DataFrame(gs.cv_results_)
        cv_csv = cv_results.to_csv(index=False)
        mlflow.log_text(cv_csv, f"cv_results/{name}_cv_results.csv")

        # Params & time
        mlflow.log_params(gs.best_params_)
        mlflow.log_metric("cv_mean_auc_pr", cv_best_score)
        mlflow.log_metric("cv_std_auc_pr", cv_results["mean_test_score"].std())
        mlflow.log_metric("train_time_sec", elapsed)

        # Save CV results
        cv_path = f"cv_results_{name}.csv"
        cv_results.to_csv(cv_path, index=False)
        mlflow.log_artifact(cv_path, artifact_path="cv_results")

        # Validation metrics + diagrams
        val_metrics = evaluate_model(best_model, X_val, y_val)
        mlflow.log_metrics({f"val_{k}": v for k, v in val_metrics.items()})

        log_diagnostic_plots(best_model, X_val, y_val, model_name=name)
        log_feature_importances(best_model, feature_names, model_name=name)

        # Log model artifact
        mlflow.sklearn.log_model(best_model, artifact_path="model")

        print(f"[{name}] Best CV AUC-PR: {cv_best_score:.3f}")
        print(f"[{name}] Val metrics: {val_metrics}")
        print(f"[{name}] Train time (sec): {elapsed:.1f}")

        return best_model, cv_best_score, val_metrics, elapsed

In [14]:
# -------------------------
# 7. Run the full model zoo
# -------------------------
all_results = []

for name, (model, grid) in models_grids.items():
    try:
        best_model, cv_score, val_metrics, elapsed = run_gridsearch_experiment(
            name=name,
            model=model,
            param_grid=grid,
            X_train=X_train,
            y_train=y_train,
            X_val=X_val,
            y_val=y_val,
            feature_names=feature_names,
            cv_splits=5,
        )

        all_results.append(
            {
                "model": name,
                "cv_mean_auc_pr": cv_score,
                "train_time_sec": elapsed,
                **{f"val_{k}": v for k, v in val_metrics.items()},
            }
        )
    except Exception as e:
        print(f"❌ {name} failed with error: {e}")
        all_results.append(
            {
                "model": name,
                "cv_mean_auc_pr": np.nan,
                "train_time_sec": np.nan,
                "val_roc_auc": np.nan,
                "val_auc_pr": np.nan,
                "val_precision": np.nan,
                "val_recall": np.nan,
                "val_f1": np.nan,
            }
        )


=== Running GridSearch for LogisticRegression ===


2025/11/08 17:46:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 17:47:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[LogisticRegression] Best CV AUC-PR: 0.773
[LogisticRegression] Val metrics: {'roc_auc': 0.9859111111111111, 'auc_pr': 0.7696705345041894, 'precision': 0.8571428571428571, 'recall': 0.6, 'f1': 0.7058823529411765}
[LogisticRegression] Train time (sec): 1.8

=== Running GridSearch for SVC ===


2025/11/08 17:47:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 17:47:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[SVC] Best CV AUC-PR: 0.737
[SVC] Val metrics: {'roc_auc': 0.9774222222222223, 'auc_pr': 0.7542442125790424, 'precision': 0.9230769230769231, 'recall': 0.4, 'f1': 0.5581395348837209}
[SVC] Train time (sec): 2.0

=== Running GridSearch for KNeighborsClassifier ===


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/ashwin/Applications/Master/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/ashwin/Applications/Master/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 140, number of negative: 3500
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001525 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1954
[LightGBM] [Info] Number of data points in the train set: 3640, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038462 -> initscore=-3.218876
[LightGBM] [Info] Start training from score -3.218876
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/Users/ashwin/Applications/Master/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/ashwin/Applications/Master/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/11/08 17:48:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 17:48:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[LGBMClassifier] Best CV AUC-PR: 0.777
[LGBMClassifier] Val metrics: {'roc_auc': 0.9839555555555556, 'auc_pr': 0.8186894037706732, 'precision': 0.85, 'recall': 0.5666666666666667, 'f1': 0.68}
[LGBMClassifier] Train time (sec): 81.0


In [16]:
comparison = pd.DataFrame(all_results)
comparison_sorted = comparison.sort_values("val_auc_pr", ascending=False)

print("\n=== Model Comparison (sorted by val AUC-PR) ===")
display(comparison_sorted)

with mlflow.start_run(run_name="model_comparison_summary"):
    comp_csv = comparison_sorted.to_csv(index=False)
    mlflow.log_text(comp_csv, "summaries/model_comparison_strong_models.csv")

print("✅ Strong-model modeling pipeline completed.")


=== Model Comparison (sorted by val AUC-PR) ===


,model,cv_mean_auc_pr,train_time_sec,val_roc_auc,val_auc_pr,val_precision,val_recall,val_f1
7,GradientBoostingClassifier,0.767684,1.495952,0.984956,0.842163,0.894737,0.566667,0.693878
6,AdaBoostClassifier,0.757235,1.140667,0.986356,0.826782,0.941176,0.533333,0.680851
13,LGBMClassifier,0.776664,81.003165,0.983956,0.818689,0.850000,0.566667,0.680000
12,CatBoostClassifier,0.791680,2.871187,0.989111,0.813278,0.490909,0.900000,0.635294
4,RandomForestClassifier,0.780644,1.496758,0.985911,0.798823,0.944444,0.566667,0.708333
8,XGBClassifier,0.781268,1.690112,0.984933,0.795094,0.473684,0.900000,0.620690
0,LogisticRegression,0.773330,1.800317,0.985911,0.769671,0.857143,0.600000,0.705882
2,KNeighborsClassifier,0.705317,0.243826,0.955156,0.768339,0.941176,0.533333,0.680851
1,SVC,0.737043,1.970476,0.977422,0.754244,0.923077,0.400000,0.558140
5,ExtraTreesClassifier,0.783728,1.214497,0.987111,0.742123,0.411765,0.933333,0.571429


✅ Strong-model modeling pipeline completed.
